## Imports, function definitions

Note: DeepMoD does not work for python3.9

In [1]:
# pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
# pip install pysindy

In [3]:
# pip install DeePyMoD

In [4]:
# pip install --upgrade tensorflow

In [5]:
# pip install tensorflow-io

In [6]:
results_path = ""

In [7]:
import matplotlib.pyplot as plt

# General imports
import numpy as np
import torch
import pandas as pd

# DeePyMoD imports
from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.data.burgers import burgers_delta
from deepymod.model.constraint import LeastSquares
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.sparse_estimators import Threshold
from deepymod.training import train
from deepymod.training.sparsity_scheduler import Periodic, TrainTest, TrainTestPeriodic

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import pickle
from sklearn.metrics import mean_squared_error
def load_data(X,y):
    return torch.from_numpy(X).float(),torch.from_numpy(y).float()

def uxt_2D_to1D(u,x,t):
    '''
    Function for reshaping u,x and t
    '''
    t_reg=[]
    x_reg=[]
    for i in range(len(t)):
        for j in range(len(x)):
            t_reg.append(t[i])
            x_reg.append(x[j])
    u_reg = u.reshape((u.size, 1))

    return u_reg,x_reg,t_reg

descr = ['',
 'u_{x}',
 'u_{xx}',
 'u_{xxx}',
 'u',
 'uu_{x}',
 'uu_{xx}',
 'uu_{xxx}',
 'u^2',
 'u^2u_{x}',
 'u^2u_{xx}',
 'u^2u_{xxx}']

def return_pde(w, rhs_description=descr, ut = 'u_t'):
    pde = ut + ' = '
    first = True
    for i in range(len(w)):
        if w[i] != 0:
            if not first:
                pde = pde + ' + '
            pde = pde + "(%05f)" % (w[i].real) + rhs_description[i] 
            first = False
    return pde


cuda


In [8]:
def error_metric(true_coeff,pred_coeff):
    wrmse = mean_squared_error(true_coeff,pred_coeff,squared=False)
    return wrmse

## Testing 1.1

In [15]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(1,3):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  20000
Dataset is using device:  cuda


2023-04-28 10:58:41.488918: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 10:58:42.001281: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 11500  MSE: 5.13e-06  Reg: 1.12e-05  L1: 1.45e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097818)u_{xx} + (-0.984390)uu_{x}  rmse =  0.004550108592672523
Dataset is using device:  cuda
 19750  MSE: 7.65e-06  Reg: 5.67e-06  L1: 1.75e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097930)u_{xx} + (-1.119017)uu_{x} + (0.264745)u^2u_{x}  rmse =  0.08379494336452206
Dataset is using device:  cuda
  9000  MSE: 3.87e-06  Reg: 4.50e-06  L1: 1.40e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098293)u_{xx} + (-0.992859)uu_{x}  rmse =  0.0021194623849890228
Dataset is using device:  cuda
 19750  MSE: 4.28e-06  Reg: 5.80e-06  L1: 1.46e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099066)u_{xx} + (-0.989616)uu_{x}  rmse =  0.0030096012957181385
Dataset is using device:  cuda
 19750  MSE: 5.76e-06  Reg: 7.53e-06  L1: 1.46e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (

KeyboardInterrupt: 

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(2,3):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  15000
Dataset is using device:  cuda


2023-04-29 01:03:07.252261: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 01:03:07.761929: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 10750  MSE: 6.26e-06  Reg: 1.23e-05  L1: 1.45e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098048)u_{xx} + (-0.982153)uu_{x}  rmse =  0.005182587198915081
Dataset is using device:  cuda
 19750  MSE: 8.27e-06  Reg: 5.66e-06  L1: 1.75e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098155)u_{xx} + (-1.115107)uu_{x} + (0.256574)u^2u_{x}  rmse =  0.08118060212396988
Dataset is using device:  cuda
  7250  MSE: 4.79e-06  Reg: 6.04e-06  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097832)u_{xx} + (-0.991981)uu_{x}  rmse =  0.0023979820230668387
Dataset is using device:  cuda
 19000  MSE: 4.95e-06  Reg: 6.04e-06  L1: 1.46e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098876)u_{xx} + (-0.990505)uu_{x}  rmse =  0.002760143553409131
Dataset is using device:  cuda
 18000  MSE: 5.91e-06  Reg: 6.00e-06  L1: 1.48e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0

## Testing 1.2

In [9]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(3,5):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  10000
Dataset is using device:  cuda
 19750  MSE: 7.26e-06  Reg: 9.42e-06  L1: 1.46e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098943)u_{xx} + (-0.990228)uu_{x}  rmse =  0.0028372638730283085
Dataset is using device:  cuda
 19750  MSE: 1.76e-05  Reg: 1.17e-05  L1: 2.08e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.096591)u_{xx} + (-1.227500)uu_{x} + (0.493390)u^2u_{x}  rmse =  0.15684434469772005
Dataset is using device:  cuda
  6250  MSE: 8.35e-06  Reg: 9.64e-06  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098788)u_{xx} + (-0.987956)uu_{x}  rmse =  0.0034944109189102117
Dataset is using device:  cuda
 19750  MSE: 6.18e-06  Reg: 6.91e-06  L1: 1.42e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099240)u_{xx} + (-0.992936)uu_{x}  rmse =  0.0020510727281364168
Dataset is using device:  cuda
 15000  MSE: 5.24e-06  Reg: 5.46e-06  L1: 1.45e+00 Algorithm converged. W

## Testing 1.3

In [9]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(5,7):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  5000
Dataset is using device:  cuda


2023-04-30 12:04:47.418874: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 12:04:47.980250: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 11750  MSE: 1.01e-05  Reg: 1.09e-05  L1: 1.45e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097586)u_{xx} + (-0.983227)uu_{x}  rmse =  0.004891790277360461
Dataset is using device:  cuda
 19750  MSE: 1.25e-05  Reg: 7.61e-06  L1: 1.44e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098888)u_{xx} + (-0.990004)uu_{x}  rmse =  0.002903333434535161
Dataset is using device:  cuda
  8750  MSE: 9.24e-06  Reg: 4.48e-06  L1: 1.44e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098768)u_{xx} + (-0.994214)uu_{x}  rmse =  0.0017078135056400234
Dataset is using device:  cuda
 19750  MSE: 9.13e-06  Reg: 5.00e-06  L1: 1.44e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098595)u_{xx} + (-0.992778)uu_{x}  rmse =  0.002124047289689097
Dataset is using device:  cuda
 19500  MSE: 9.99e-06  Reg: 6.50e-06  L1: 1.51e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098969)u_{xx} + (-0

## Testing 1.4

In [10]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(7,9):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  2000
Dataset is using device:  cuda
 11750  MSE: 2.18e-05  Reg: 9.86e-06  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098125)u_{xx} + (-0.986039)uu_{x}  rmse =  0.004066486378375345
Dataset is using device:  cuda
 12000  MSE: 2.18e-05  Reg: 1.30e-05  L1: 1.38e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099526)u_{xx} + (-0.982764)uu_{x}  rmse =  0.004977434497949543
Dataset is using device:  cuda
  6000  MSE: 2.47e-05  Reg: 1.33e-05  L1: 1.46e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099529)u_{xx} + (-0.984345)uu_{x}  rmse =  0.004521166002016839
Dataset is using device:  cuda
  6500  MSE: 2.23e-05  Reg: 1.09e-05  L1: 1.81e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.094916)u_{xx} + (-1.134438)uu_{x} + (0.308943)u^2u_{x}  rmse =  0.09727338250862608
Dataset is using device:  cuda
  9500  MSE: 2.22e-05  Reg: 1.09e-05  L1: 1.46e+00 Algorithm converged. Writi

KeyboardInterrupt: 

In [9]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(8,9):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  1000
Dataset is using device:  cuda


2023-05-03 10:12:10.167093: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 10:12:10.736557: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 10000  MSE: 3.81e-05  Reg: 1.20e-05  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.033669)u_{x} + (0.093523)u_{xx} + (-0.895520)uu_{x}  rmse =  0.03174314697143666
Dataset is using device:  cuda
 19250  MSE: 3.67e-05  Reg: 6.83e-06  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099898)u_{xx} + (-0.991088)uu_{x}  rmse =  0.0025727109606493203
Dataset is using device:  cuda
  5500  MSE: 3.72e-05  Reg: 1.23e-05  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.096484)u_{xx} + (-0.984447)uu_{x}  rmse =  0.004603027231601286
Dataset is using device:  cuda
 18500  MSE: 3.48e-05  Reg: 6.45e-06  L1: 1.46e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098461)u_{xx} + (-0.990812)uu_{x}  rmse =  0.00268943978545998
Dataset is using device:  cuda
 19750  MSE: 3.62e-05  Reg: 7.12e-06  L1: 1.47e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.09

## Testing 1.5

In [9]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(9,11):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  800
Dataset is using device:  cuda


2023-05-05 10:53:29.879951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 10:53:30.461107: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  9750  MSE: 4.55e-05  Reg: 1.17e-05  L1: 1.44e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.032087)u_{x} + (0.093713)u_{xx} + (-0.903051)uu_{x}  rmse =  0.029535538609983442
Dataset is using device:  cuda
 18250  MSE: 4.52e-05  Reg: 7.77e-06  L1: 1.45e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.100301)u_{xx} + (-0.989789)uu_{x}  rmse =  0.0029490735477482673
Dataset is using device:  cuda
  6750  MSE: 4.43e-05  Reg: 8.21e-06  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097408)u_{xx} + (-0.987602)uu_{x}  rmse =  0.0036563948446178547
Dataset is using device:  cuda
 19750  MSE: 4.25e-05  Reg: 3.72e-06  L1: 1.68e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098309)u_{xx} + (-1.100350)uu_{x} + (0.231623)u^2u_{x}  rmse =  0.07287091359106386
Dataset is using device:  cuda
 17500  MSE: 4.61e-05  Reg: 8.47e-06  L1: 1.49e+00 Algorithm converged. Writing model to disk.
Pred